<a href="https://colab.research.google.com/github/Seoyeon-An/YBIGTA_Can_Classification/blob/model/yolo/YOLOv8_trial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install ultralytics
from ultralytics import YOLO

In [1]:
# check if selected GPU is available or use CPU
import torch
if torch.cuda.is_available():
    print('Good to go! You are using', torch.cuda.get_device_name())
    device = torch.device('cuda')
else:
    print('Please set GPU via Edit -> Notebook Settings.')
    device = torch.device('cpu')
     
print('Default torch.dtype is:', torch.get_default_dtype())

Good to go! You are using Tesla T4
Default torch.dtype is: torch.float32


## Constructing Dataset

In [2]:
#Connect Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!unzip -q /content/drive/MyDrive/ybigta/2023-1/Conference/Total_can_drink.zip -d /content/Dataset

In [4]:
import os
import random
from sklearn.model_selection import train_test_split
import shutil

# Set the path to your image dataset folder
dataset_folder = "/content/Dataset/download"

# Set the path for the train and test folders
train_folder = "/content/CanDataset/train"
test_folder = "/content/CanDataset/test"
val_folder = "/content/CanDataset/val"

# Set the random seed for reproducibility
random.seed(42)

# Get the list of subfolders (image classes) in the dataset folder
subfolders = [f.path for f in os.scandir(dataset_folder) if f.is_dir()]

# Create train and test folders
os.makedirs(train_folder, exist_ok=True)
os.makedirs(test_folder, exist_ok=True)
os.makedirs(val_folder, exist_ok=True)

# Iterate over each subfolder and split the images
for subfolder in subfolders:
    class_label = os.path.basename(subfolder)
    images = [f.path for f in os.scandir(subfolder) if f.is_file()]
    
    # Split the images into train and test sets
    train_images, test_images = train_test_split(images, test_size=0.2, random_state=42)
    train_images, val_images = train_test_split(train_images, test_size = 0.2, random_state = 42)
    
    # Create train and test folders for the class
    train_class_folder = os.path.join(train_folder, class_label)
    test_class_folder = os.path.join(test_folder, class_label)
    val_class_folder = os.path.join(val_folder, class_label)
    os.makedirs(train_class_folder, exist_ok=True)
    os.makedirs(test_class_folder, exist_ok=True)
    os.makedirs(val_class_folder, exist_ok=True)
    
    # Move the train images to the train folder
    for image in train_images:
        shutil.copy2(image, train_class_folder)
    
    # Move the test images to the test folder
    for image in test_images:
        shutil.copy2(image, test_class_folder)

    # Move the validation images to the val folder
    for image in test_images:
        shutil.copy2(image, val_class_folder)


In [8]:
import yaml

data_dict = {
    "train": "/content/CanDataset/train",
    "val": "/content/CanDataset/val",
    "test": "/content/CanDataset/test",
}

with open("/content/CanDataset/data.yaml", 'w') as f:
  yaml.dump(data_dict, f)

In [9]:
colab_folder = '/content/CanDataset'
drive_folder = '/content/drive/MyDrive/ybigta/2023-1/Conference/'  # Replace with the desired destination folder in Google Drive

zip_file = f'{drive_folder}/CanDataset'  # Replace with the desired name for the zip file

shutil.make_archive(zip_file, 'zip', colab_folder)

'/content/drive/MyDrive/ybigta/2023-1/Conference/CanDataset.zip'

In [3]:
!unzip -q /content/drive/MyDrive/ybigta/2023-1/Conference/CanDataset.zip -d /content/CanDataset

## Model

In [7]:
model = YOLO('yolov8n-cls.pt') 

model.train(data='/content/CanDataset/', epochs=10, imgsz=32)

Ultralytics YOLOv8.0.108 🚀 Python-3.10.11 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
yolo/engine/trainer: task=classify, mode=train, model=yolov8n-cls.pt, data=/content/CanDataset/, epochs=10, patience=50, batch=16, imgsz=(32, 32), save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=False, int8=False, dynamic=False, simplify=False, 